In [ ]:
# install the requirements
# !pip install gensim

# Ejemplo de `word2vec` y `fastText` con `gensim`

En la siguiente celda, importamos las librerías necesarias y configuramos los mensajes de los logs.

In [1]:
import gensim, logging, os

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)

## Entrenamiento de un modelo

Implemento una clase `Corpus` con un iterador sobre un directorio que contiene ficheros de texto. Utilizaré una instancia de `Corpus` para poder procesar de manera más eficiente una colección, sin necesidad de cargarlo previamente en memoria.

In [ ]:
class Corpus(object):
    """Clase Corpus que permite leer de manera secuencial un directorio de documentos de texto"""

    def __init__(self, directorio):
        self.directory = directorio

    def __iter__(self):
        for fichero in os.listdir(self.directory):
            for linea in open(os.path.join(self.directory, fichero)):
                yield linea.split()

`CORPUSDIR` contiene una colección de noticias en español (normalizada previamente a minúsculas y sin signos de puntuación) con alrededor de 150 millones de palabras. Entrenamos un modelo en un solo paso, ignorando aquellos tokens que aparecen menos de 10 veces (ignorando erratas, básicamente), para construir vectores de 200 dimensiones. 

In [ ]:
CORPUSDIR = "PATH_TO_YOUR_CORPUS_DIRECTORY"
oraciones = Corpus(CORPUSDIR)
model = gensim.models.Word2Vec(oraciones, min_count=10, size=200, workers=2)

Una vez completado el entrenamiento (después de casi 30 minutos), guardamos el modelo en disco. 

In [ ]:
model.save("PATH_TO_YOUR_MODEL.w2v")

En el futuro, podremos utilizar este modelo cargándolo en memoria con la instrucción:

In [2]:
model = gensim.models.Word2Vec.load("../data/eswiki-300.w2v")

2021-01-31 15:39:35,005 : INFO : loading Word2Vec object from ../data/eswiki-300.w2v
2021-01-31 15:39:36,874 : INFO : loading trainables recursively from ../data/eswiki-300.w2v.trainables.* with mmap=None
2021-01-31 15:39:36,875 : INFO : loading syn1neg from ../data/eswiki-300.w2v.trainables.syn1neg.npy with mmap=None
2021-01-31 15:39:37,242 : INFO : loading vocabulary recursively from ../data/eswiki-300.w2v.vocabulary.* with mmap=None
2021-01-31 15:39:37,243 : INFO : loading wv recursively from ../data/eswiki-300.w2v.wv.* with mmap=None
2021-01-31 15:39:37,243 : INFO : loading vectors from ../data/eswiki-300.w2v.wv.vectors.npy with mmap=None
2021-01-31 15:39:37,593 : INFO : setting ignored attribute vectors_norm to None
2021-01-31 15:39:37,594 : INFO : setting ignored attribute cum_table to None
2021-01-31 15:39:37,595 : INFO : loaded ../data/eswiki-300.w2v


## Probando nuestro modelo

El objeto `model` contiene una enorme matriz de números: una tabla, donde cada fila es uno de los términos del vocabulario reconocido y cada columna es una de las características que permiten modelar el significado de dicho término.

En nuestro modelo, tal y como está entrenado, tenemos más de 41 millones de términos:

In [3]:
print(model.corpus_count)

41843901


Cada término del vocabulario está representado como un vector con 150 dimensiones: 105 características. Podemos acceder al vector de un término concreto:

In [4]:
print(model["azul"])
print(model["verde"])
print(model["clorofila"])

[ 6.97230875e-01 -2.16513014e+00  3.48829389e-01 -9.89584625e-01
 -1.02388668e+00 -2.17564240e-01 -1.96011472e+00  2.94649076e+00
 -9.12335694e-01 -6.84771001e-01  1.52577829e+00  7.75085762e-02
 -7.02408105e-02  4.89242375e-01  1.37050912e-01 -9.30723965e-01
  6.80433512e-02 -1.08530678e-01  2.31580353e+00 -5.53485096e-01
 -3.57593626e-01 -2.28935838e+00 -1.50510311e-01  1.26760174e-02
  6.07259810e-01 -2.17627048e+00  2.73906016e+00  1.09757566e+00
 -1.71337926e+00  1.64041713e-01  6.90737784e-01  1.26919913e+00
 -5.12029469e-01 -2.01885894e-01  1.78642869e-01  1.53002536e+00
 -1.43445313e-01  1.11213720e+00 -1.41976345e+00 -1.72563398e+00
  7.15158105e-01  1.11852288e+00 -5.18750012e-01 -7.85638809e-01
 -2.75143218e+00 -2.21702838e+00 -7.32014835e-01 -4.00392205e-01
  1.58708438e-01  1.02694380e+00 -2.80862898e-01  1.41663051e+00
  1.11722755e+00 -1.24235370e-03 -1.14149344e+00 -1.97176194e+00
 -2.22860837e+00 -1.23289347e+00  1.74586341e-01  4.37671602e-01
 -9.38071907e-01  4.78671

<ipython-input-4-463719f921d6>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  print(model["azul"])
<ipython-input-4-463719f921d6>:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  print(model["verde"])
<ipython-input-4-463719f921d6>:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  print(model["clorofila"])


Estos vectores no nos dicen mucho, salvo que contienen números muy pequeños :-/

El mismo objeto `model` permite acceder a una serie de funcionalidades ya implementadas que nos van a permitir evaluar formal e informalmente el modelo. Por el momento, nos contentamos con los segundo: vamos a revisar visualmente los significados que nuestro modelo ha aprendido por su cuenta. 

Podemos calcular la similitud semántica entre dos términos usando el método `similarity`, que nos devuelve un número entre 0 y 1:

In [5]:
print("hombre - mujer", model.wv.similarity("hombre", "mujer"))

print("gato - mujer", model.wv.similarity("gato", "mujer"))

print("perro - gato", model.wv.similarity("perro", "gato"))

print("gato - periódico", model.wv.similarity("gato", "periódico"))

print("febrero - azul", model.wv.similarity("febrero", "azul"))

hombre - mujer 0.4308716
gato - mujer 0.261357
perro - gato 0.8005251
gato - periódico 0.17121993
febrero - azul -0.021135565


Podemos seleccionar el término que no encaja a partir de una determinada lista de términos usando el método `doesnt_match`:

In [6]:
lista1 = "madrid barcelona gonzález washington".split()
print("en la lista", " ".join(lista1), "sobra:", model.wv.doesnt_match(lista1))

lista2 = "psoe pp ciu ronaldo".split()
print("en la lista", " ".join(lista2), "sobra:", model.wv.doesnt_match(lista2))

lista3 = "publicaron declararon soy negaron".split()
print("en la lista", " ".join(lista3), "sobra:", model.wv.doesnt_match(lista3))

lista4 = "homero saturno cervantes shakespeare cela".split()
print("en la lista", " ".join(lista4), "sobra:", model.wv.doesnt_match(lista4))

lista5 = "madrid barcelona alpedrete marsella".split()
print("en la lista", " ".join(lista5), "sobra:", model.wv.doesnt_match(lista5))

2021-01-31 15:39:52,221 : INFO : precomputing L2-norms of word weight vectors


en la lista madrid barcelona gonzález washington sobra: washington
en la lista psoe pp ciu ronaldo sobra: ronaldo
en la lista publicaron declararon soy negaron sobra: soy
en la lista homero saturno cervantes shakespeare cela sobra: saturno
en la lista madrid barcelona alpedrete marsella sobra: alpedrete


/home/victor/miniconda3/envs/nlp/lib/python3.8/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


Podemos buscar los términos más similares usando el método `most_similar` de nuestro modelo:

In [7]:
terminos = "azul madrid bmw bici 2018 rock google psoe jay-z xiaomi rajoy brexit saturno césar lazio".split()

for t in terminos:
    print(t, "==>", model.wv.most_similar(t), "\n")

azul ==> [('verde', 0.7170504331588745), ('amarillo', 0.7170424461364746), ('rojo', 0.7002288699150085), ('azúl', 0.6983712315559387), ('turquesa', 0.6890928745269775), ('morado', 0.6765597462654114), ('carmesí', 0.6581361293792725), ('granate', 0.6548364162445068), ('naranja', 0.6507274508476257), ('beige', 0.6397430896759033)] 

madrid ==> [('valladolid', 0.7847391366958618), ('barcelona', 0.7758660316467285), ('sevilla', 0.760832667350769), ('zaragoza', 0.7517766952514648), ('madrid.', 0.7219818830490112), ('valencia', 0.6873288154602051), ('oviedo', 0.6831979751586914), ('salamanca', 0.6781077980995178), ('málaga', 0.6757391095161438), ('pamplona', 0.6748294830322266)] 

bmw ==> [('audi', 0.7591230273246765), ('porsche', 0.7583965063095093), ('mercedes-benz', 0.7572088241577148), ('renault', 0.7485952973365784), ('toyota', 0.7476786375045776), ('opel', 0.739383339881897), ('volkswagen', 0.7309398651123047), ('maserati', 0.7237600088119507), ('gt', 0.7185136675834656), ('volvo', 0.7

Con el mismo método `most_similar` podemos combinar vectores de palabras tratando de jugar con los rasgos semánticos de cada una de ellas para descubrir nuevas relaciones.

In [8]:
print("mujer que ejerce la autoridad en una alcaldía ==> alcalde + mujer - hombre")
most_similar = model.wv.most_similar(
    positive=["alcalde", "mujer"], negative=["hombre"], topn=3
)
for item in most_similar:
    print(item)

print(
    "mujer especializada en alguna terapia de la medicina ==> doctor + mujer - hombre"
)
most_similar = model.wv.most_similar(
    positive=["doctor", "mujer"], negative=["hombre"], topn=3
)
for item in most_similar:
    print(item)

print("monarca soberano ==> reina + hombre - mujer")
most_similar = model.wv.most_similar(
    positive=["reina", "hombre"], negative=["mujer"], topn=3
)
for item in most_similar:
    print(item)

print("capital de Alemania ==> moscú + alemania - rusia")
most_similar = model.wv.most_similar(
    positive=["moscú", "alemania"], negative=["rusia"], topn=3
)
for item in most_similar:
    print(item)

print("presidente de Francia ==> obama + francia - eeuu")
most_similar = model.wv.most_similar(
    positive=["obama", "francia"], negative=["eeuu"], topn=3
)
for item in most_similar:
    print(item)

mujer que ejerce la autoridad en una alcaldía ==> alcalde + mujer - hombre
('alcaldesa', 0.7252961993217468)
('regidora', 0.6073892116546631)
('concejala', 0.5990710258483887)
mujer especializada en alguna terapia de la medicina ==> doctor + mujer - hombre
('doctora', 0.6626786589622498)
('enfermera', 0.5988627076148987)
('esposa', 0.5201101303100586)
monarca soberano ==> reina + hombre - mujer
('rey', 0.6301776170730591)
('monarca', 0.5567039251327515)
('príncipe', 0.5296465158462524)
capital de Alemania ==> moscú + alemania - rusia
('berlín', 0.7966122031211853)
('múnich', 0.7589867115020752)
('hamburgo', 0.7321968078613281)
presidente de Francia ==> obama + francia - eeuu
('hollande', 0.4829712212085724)
('papado', 0.45290911197662354)
('napoleón', 0.4527309536933899)


## Probando el modelo de `fastText`

También cargamos los vectores de `fastText` entrenados por Facebook AI.

La siguiente celda carga los vectores que hemos descargado y que están en formato texto. Esto implica que tardará casi 10 minutos en parsear el fichero completo.

In [11]:
fastText = gensim.models.KeyedVectors.load_word2vec_format("../data/cc.en.300.vec")

2021-01-31 15:40:11,314 : INFO : loading projection weights from ../data/cc.en.300.vec
2021-01-31 15:49:11,829 : INFO : loaded (2000000, 300) matrix from ../data/cc.en.300.vec


In [12]:
terminos = "blue Madrid BMW bike 2018 rock google Google Jay-Z Xiaomi Brexit Saturn Lazio ".split()

for t in terminos:
    print(t, "==>", fastText.wv.most_similar(t), "\n")

<ipython-input-12-14b384a0e4ac>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  print(t, "==>", fastText.wv.most_similar(t), "\n")
2021-01-31 15:49:42,676 : INFO : precomputing L2-norms of word weight vectors


blue ==> [('red', 0.8113019466400146), ('purple', 0.8068118095397949), ('yellow', 0.7786766886711121), ('pink', 0.7204939126968384), ('light-blue', 0.7144668102264404), ('green-blue', 0.7136971950531006), ('green', 0.7049668431282043), ('orange', 0.7043564319610596), ('blue-green', 0.7041903734207153), ('white', 0.6973941326141357)] 

Madrid ==> [('Barcelona', 0.8364297151565552), ('Sevilla', 0.7913062572479248), ('Madrid.The', 0.7868168950080872), ('Madrid-', 0.7595229744911194), ('Zaragoza', 0.7440358400344849), ('Madrid.', 0.7405192852020264), ('Spain', 0.7340582013130188), ('Malaga', 0.734052836894989), ('Málaga', 0.7251549959182739), ('Valencia', 0.724647045135498)] 

BMW ==> [('Audi', 0.7336082458496094), ('bmw', 0.7096737027168274), ('Bmw', 0.7082890272140503), ('BMWs', 0.7048546075820923), ('Porsche', 0.6972150802612305), ('3-series', 0.6832477450370789), ('320i', 0.6801005601882935), ('Z4s', 0.6781718730926514), ('non-BMW', 0.6766112446784973), ('730Ld', 0.672356367111206)] 



In [17]:
most_similar = model.wv.most_similar(
    positive=["doctor", "woman"], negative=["man"], topn=3
)
for item in most_similar:
    print(item)
print("-"*30)

most_similar = model.wv.most_similar(
    positive=["queen", "man"], negative=["woman"], topn=3
)
for item in most_similar:
    print(item)
print("-"*30)

most_similar = model.wv.most_similar(
    positive=["moscow", "germany"], negative=["russia"], topn=3
)
for item in most_similar:
    print(item)
print("-"*30)

most_similar = model.wv.most_similar(
    positive=["obama", "france"], negative=["usa"], topn=3
)
for item in most_similar:
    print(item)

('dr.', 0.5067815184593201)
('doctora', 0.4895417392253876)
('psiquiatra', 0.4575495421886444)
------------------------------
('king', 0.5242592096328735)
('duke', 0.4742013216018677)
('maiden', 0.44714492559432983)
------------------------------
('vienna', 0.6613255143165588)
('prague', 0.6580424308776855)
('berlin', 0.6535778045654297)
------------------------------
('hollande', 0.5331801772117615)
('essebsi', 0.5026109218597412)
('sarkozy', 0.4846283793449402)
